In [ ]:
from sympy import eye, symbols, Matrix
from SymE3.core import Plane, LieGroup, PointH, Pixel, LieAlgebra, exp, CustomFunction, TotalFunction, dehom, Point

def show(v):
    display(v)
    if hasattr(v, 'shape'):
        print(v.shape)
    print(type(v))
    
T_cw = LieGroup("{T_{cw}}")
T_ct = LieGroup("{\hat{T}_{ct}}")
p_t = PointH("{p_t}")
phat_c = PointH("{\hat{p}_{c}}")
p_c = Pixel("{p_c}")
N_w = Plane("{N_w}")
d = LieAlgebra("{\\delta}")

def proj(p):
    p_ray = p / p[2, 0]
    f_x, f_y, c_x, c_y = symbols("f_x f_y c_x c_y")
    
    return Matrix([[f_x,   0, c_x],
                   [  0, f_y, c_y]]) * p_ray

Pi = CustomFunction("Pi", proj, 3, 2)

def sym(n):
    n_hat = n[0:3, :]
    S = eye(4)
    S[0:3, 0:3] = eye(3) - (2 * (n_hat * n_hat.transpose()))
    S[0:3, 3] = 2 * n[3] * n_hat
    return S
    
S = CustomFunction("S", sym, 4, 4, 1, 4)

e = Pi(dehom(T_cw * S(N_w) * T_cw.inverse() * exp(d) * T_ct * p_t)) - p_c
print("Function:")
show(e)
e = e.subs(T_ct * p_t, phat_c)

f = TotalFunction(e)

print("Substituted:")
show(f)

print("Expanded:")
fe = f.as_explicit()
phat_se = TotalFunction(dehom(T_cw * S(N_w) * T_cw.inverse() * exp(d) * phat_c)).as_explicit()
phat_s = TotalFunction(Point("{\\hat{p}_s}")).as_explicit()

for row in range(3):
    fe = fe.subs(phat_se[row, 0], phat_s[row, 0])
show(fe)

print("Derivative:")
df_dd = f.diff(d, N_w)

S_ce = TotalFunction(T_cw * S(N_w) * T_cw.inverse()).as_explicit()
S_c = TotalFunction(LieGroup("{S_c}")).as_explicit()
ds_dne = TotalFunction(T_cw * S(N_w) * T_cw.inverse() * phat_c).diff(N_w)
ds_dn = TotalFunction(LieGroup("{\partial}S(N_w)")).as_explicit()
                                                                    
for row in range(3):
    df_dd = df_dd.subs(phat_se[row, 0], phat_s[row, 0])
    for col in range(4):
        df_dd = df_dd.subs(S_ce[row, col], S_c[row, col])
        df_dd = df_dd.subs(ds_dne[row, col], ds_dn[row, col])

show(df_dd)

Function:


pi - {p_c}

(2, 1)
<class 'sympy.matrices.expressions.matadd.MatAdd'>
Substituted:


<class 'SymE3.core.TotalFunction'>
Expanded:


[[c_x + f_x*{{\hat{p}_s}_{0}}/{{\hat{p}_s}_{2}} - {{p_c}_{0}}], [c_y + f_y*{{\hat{p}_s}_{1}}/{{\hat{p}_s}_{2}} - {{p_c}_{1}}]]

(2, 1)
<class 'SymE3.detail._Explicit'>
Derivative:
